In [1]:

import pyodbc
import jieba

from math import*
import codecs
from nltk import word_tokenize
import os #讀取程式目前路徑用
import sys #為了能夠吃進主程式外部輸進的參數
import nltk #引入自然語言處理模組
import io #檔案io
from nltk.corpus import stopwords #去無用字
import re #需要用正則表示式移除標點與數字
from nltk.corpus import wordnet as wn #2013/10/26改用wordnet來做字根還原
from nltk.tokenize import word_tokenize
import json
import numpy as np
import numpy.linalg as LA
import string
import csv
from __future__ import print_function, unicode_literals
import sys
sys.path.append("../")
import jieba
import jieba.posseg as pseg

print('create all the dictionary and list....')
###建立同義詞詞典(戴老師指定:key為完整編號，value為所有詞)
dict_data_full_number_policy = {}
dict_key_2 = {}
dict_key_2['whatever']="just_for_test"
with open('full_number_policy.txt' , encoding = 'utf8') as df:
    for kv in [d.strip().split('	') for d in df]:
        #print('kv[0]=',kv[0])
        if kv[0] not in dict_key_2['whatever']:
            dict_key_2['whatever'] = kv[0]      
        #print('kv[1]=',kv[1])
        #print()
        dict_data_full_number_policy.setdefault(kv[0], [])
        dict_data_full_number_policy[kv[0]].append(kv[1])

###建立同義詞詞典(戴老師指定:key為所有詞，value為同義詞編號)
dict_data_key_in_policy = {}
dict_key_1 = {}
dict_key_1['whatever']="just_for_test"
with open('key_in_policy.txt' , encoding = 'utf8') as df:
    for kv in [d.strip().split('	') for d in df]:
        #print('kv[0]=',kv[0])
        if kv[0] not in dict_key_1['whatever']:
            dict_key_1['whatever'] = kv[0]      
        #print('kv[1]=',kv[1])
        #print()
        dict_data_key_in_policy.setdefault(kv[0], [])
        dict_data_key_in_policy[kv[0]].append(kv[1])

###建立同義詞詞典
dict_data = {}
dict_key = {}
dict_key['abc']="aaaaaa"
with open('split_cate.txt' , encoding = 'utf8') as df:
    for kv in [d.strip().split('	') for d in df]:
        #print('kv[0]=',kv[0])
        if kv[0] not in dict_key['abc']:
            dict_key['abc'] = kv[0]      
        #print('kv[1]=',kv[1])
        #print()
        dict_data.setdefault(kv[0], [])
        dict_data[kv[0]].append(kv[1])
        
aaa='A0707\nA2203\nA1402\nA0705\nA2202\nA2313\nA0711\nA0203\nA2602\nA0101\nA0713\nA1504\nA1906\nA1506\nA2604\nA2314\nA1705\nA2321\nA2408\nA1702\nA0403\nA2309\nA2325\nA0621\nA2101\nA2407\nA2002\nA2711\nA1905\nA2503\nA2703\nA0906\nA0704\nA1202\nA0505\nA2902\nA0626\nA2509\nA2310\nA1902\nA3101\nA2418\nA0604\nA1803\nA0607\nA0625\nA2205\nA0801\nA2710\nA0703\nA2718\nA2403\nA2415\nA0714\nA2716\nA2304\nA2606\nA1007\nA2312\nA3102\nA2701\nA1801\nA1401\nA0401\nA0605\nA2712\nA1203\nA2417\nA0623\nA2416\nA0708\nA0504\nA0502\nA0202\nA1508\nA2324\nA1505\nA0622\nA2502\nA1512\nA1008\nA2720\nA1802\nA2507\nA0620\nA2705\nA1201\nA2904\nA3103\nA2802\nA0102\nA1406\nA1407\nA1502\nA2504\nA2302\nA2406\nA2003\nA0614\nA0802\nA2104\nA2204\nA0610\nA0506\nA0501\nA2103\nA0402\nA2317\nA2419\nA0615\nA2108\nA1507\nA1404\nA2201\nA1602\nA2605\nA0613\nA2501\nA0619\nA2315\nA1403\nA2401\nA2301\nA2106\nA0903\nA2702\nA2323\nA2414\nA2305\nA2206\nA2105\nA0507\nA2717\nA1004\nA1601\nA0715\nA2001\nA1002\nA2413\nA2506\nA1703\nA1701\nA2402\nA0902\nA2322\nA1005\nA2303\nA2316\nA2409\nA1901\nA0712\nA1510\nA1301\nA0612\nA1501\nA2107\nA2901\nA2704\nA0709\nA1603\nA3001\nA1101\nA2903\nA2318\nA2405\nA2410\nA0608\nA2308\nA0609\nA2412\nA0601\nA2510\nA2306\nA1509\nA0901\nA1302\nA0301\nA2601\nA0907\nA0602\nA0706\nA2319\nA1704\nA2004\nA0618\nA0201\nA2607\nA2505\nA2420\nA1511\nA1503\nA1003\nA1804\nA2307\nA1405\nA2411\nA0702\nA0616\nA2326\nA1001\nA1904\nA0905\nA1604\nA2512\nA2803\nA2708\nA0904\nA0503\nA1006\nA0710\nA2714\nA0624\nA2719\nA2320\nA1303\nA2715\nA2404\nA2804\nA2707\nA2706\nA2713\nA2511\nA0617\nA2801\nA0611\nA2508\nA2311\nA0204\nA1605\nA2709\nA2603\nA1903\nA0606\nA0603\nA2102\nA0701'
dict_data_key = aaa.split('\n')
dict_data_key.sort()

f= open('diction_split.txt', encoding = 'utf8')#讀所有詞
s = f.read()
#print(s)
spilt_s = s.split("\n")#分所有詞
del spilt_s[287] #刪最後一個空的
print('*sccessful create the custom dictionary*')

#連接自定義字典
print('import the customer dictionary....')
jieba.load_userdict("dict_policy.txt")
print('*sccessful create all the dictionary and list*')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Leaves\AppData\Local\Temp\jieba.cache


create all the dictionary and list....
*sccessful create the custom dictionary*
import the customer dictionary....


Loading model cost 0.955 seconds.
Prefix dict has been built succesfully.


*sccessful create all the dictionary and list*


In [2]:
##連結資料庫

print("try to link with DataBase:cate_test......")
cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=LEAVES-PC\SA000;DATABASE=cate_test;UID=sa;PWD=00000000")
cursor = cnxn.cursor()
print("*sccessful link*")

try to link with DataBase:cate_test......
*sccessful link*


In [24]:
#選擇資料集

cursor.execute("select [BulletinPlatform] from [cate_test].[dbo].[X_loser_candinates_20161020001]" )
#建立資料集
print('setting up the data.......')
results_1 = cursor.fetchall()
print('*sccessful set up*')

cursor.execute("select [BulletinPlatform] from [cate_test].[dbo].[X_loser_candinates_20161020001]" )
#建立資料集
print('setting up the data.......')
results_2 = cursor.fetchall()
print('*sccessful set up*')

setting up the data.......
*sccessful set up*
setting up the data.......
*sccessful set up*


In [25]:


def delpunc_n_jieba_n_tok(re): 
    count=0
    for i in re:
        count+=1
        #print ('%d: %s' % (count, i))
    for times in range(0,count):
        chineseFilter1 =  [u'．',u'的',u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
        for chin in chineseFilter1:
            re[times] = str(re[times])#轉為字串才能處理
            re[times] = re[times].replace(chin,'')
        re_cut = jieba.cut(re[times])
        re_cut_show = " ".join(re_cut)##空白一定要有不然她無法斷後面的token
        re_token = nltk.word_tokenize(re_cut_show)
        re[times] = re_token

delpunc_n_jieba_n_tok(results_1)
delpunc_n_jieba_n_tok(results_2)

In [26]:


def new_gene_list(new_gene_list_r):
    new_gene=[]
    for cani_num in range(0,len(new_gene_list_r)):
        #print('now cadina=',new_gene_list_r[cani_num])
        tem_gene=[]
        for i in range(0,len(new_gene_list_r[cani_num])):
                for j in range(0,len(spilt_s)):
                    if spilt_s[j] == new_gene_list_r[cani_num][i]:
                        tem_gene.append(spilt_s[j])
        new_gene.append(tem_gene)
        #print('現在的new_gene->',new_gene)
    return new_gene

new_gene_1 = new_gene_list(results_1)
new_gene_2 = new_gene_list(results_2)

In [27]:
def count_frequent_of_key(k_li):
    key_freq_list=[]
    for k_li_num in k_li:
        k_li_tem_freq = nltk.FreqDist(k_li_num)
        key_freq_list.append(k_li_tem_freq)
    return key_freq_list 
        
key_freq_list_1 = count_frequent_of_key(new_gene_1)
key_freq_list_2 = count_frequent_of_key(new_gene_2)

In [28]:

def convert_todict_freq_to_dict(k_f_list):
    ues_this_k_real_list=[]
    for k_f_list_dic_freq in range(0,len(k_f_list)):
        k_real_freq={}
        k_real_freq.update(k_f_list[k_f_list_dic_freq])
        ues_this_k_real_list.append(k_real_freq)
    return ues_this_k_real_list
        
ues_this_k_real_list_1 = convert_todict_freq_to_dict(key_freq_list_1)
ues_this_k_real_list_2 = convert_todict_freq_to_dict(key_freq_list_2)


In [29]:

def convert_word_to_number_in_key(word_list):
    key_w='0'
    BKUP_each_key_in_each_list='0'
    real_list_remove_for_freq = []
    real_list_remove_for_freq_1 = []
    real_remove_pun_for_freq = {}
    for k_list_in_word in word_list:
        for each_key_in_each_list in k_list_in_word:
            BKUP_each_key_in_each_list = each_key_in_each_list
            if each_key_in_each_list in dict_data_key_in_policy.keys():
                each_k_repea = str(dict_data_key_in_policy[each_key_in_each_list])
                real_remove_pun_for_freq.setdefault(each_k_repea[2:7],[])#弄個List的容器給她裝整數
                if each_k_repea[2:7] in real_remove_pun_for_freq.keys():
                    real_remove_pun_for_freq[each_k_repea[2:7]].append(k_list_in_word[BKUP_each_key_in_each_list])###注意迴圈k_list_in_word
                else:
                    real_remove_pun_for_freq.update({each_k_repea[2:7]:k_list_in_word[BKUP_each_key_in_each_list]})
            else:
                print('pass')
        backup_real_remove_pun_for_freq = real_remove_pun_for_freq.copy()    ####複製.copy()
        #print('backup_real_remove_pun_for_freq->',backup_real_remove_pun_for_freq)
        real_remove_pun_for_freq.clear()
        real_list_remove_for_freq_1.append(backup_real_remove_pun_for_freq)
    return real_list_remove_for_freq_1

real_list_remove_for_freq_1_1 = convert_word_to_number_in_key(ues_this_k_real_list_1)              
real_list_remove_for_freq_1_2 = convert_word_to_number_in_key(ues_this_k_real_list_2)


In [30]:
###把>1的數字全部變成1
def turn_num_all_to_1(real_list_remove_for_freq_1):
    for swp_real_list_remove in real_list_remove_for_freq_1:
        for swp_key_find in swp_real_list_remove.keys() :
            for num_loop_in_swp_real in range(0,len(swp_real_list_remove[swp_key_find])):
                if swp_real_list_remove[swp_key_find][num_loop_in_swp_real] >1:##在第幾個因為他每個數字都是list
                    swp_real_list_remove[swp_key_find][num_loop_in_swp_real] = 1
    return real_list_remove_for_freq_1

real_list_turn_num_all_to_1_1 = turn_num_all_to_1(real_list_remove_for_freq_1_1)
real_list_turn_num_all_to_1_2 = turn_num_all_to_1(real_list_remove_for_freq_1_2)

In [31]:
##弄向量出來

def generate_vector(g_v):
    t_fina_vector = []
    t_buff_fina_vector=[]
    for n_g_v in g_v:
        for dic_data_num in dict_data_key:
            if dic_data_num in n_g_v.keys():
                t_buff_fina_vector.append(n_g_v[dic_data_num][0])
            else:
                t_buff_fina_vector.append(0)
        copy_t_buff_fina_vector = t_buff_fina_vector[:]
        t_fina_vector.append(copy_t_buff_fina_vector)
        t_buff_fina_vector[:] = []
    return t_fina_vector

vector_1 = generate_vector(real_list_turn_num_all_to_1_1)
vector_2 = generate_vector(real_list_turn_num_all_to_1_2)

In [32]:
from scipy import spatial
import numpy as np
import numpy.linalg as LA

#dataSet = [[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
def math_fuction_of_simi(vec_c_1,vec_c_2):
    vec_consin_result = 1 - spatial.distance.cosine(vec_c_1, vec_c_2)
    #print('in math_fuction_of_simi', vec_consin_result)
    vec_consin_result = round(vec_consin_result,4)
    return vec_consin_result

f_txt = open('x_x_try.txt', 'w', encoding = 'UTF-8')


def write_simi_in_txt(ready_in_vec):
    data = (ready_in_vec)
    #print('type是',type(data))
    #print(data)
    f_txt.write(str(data))
    f_txt.write('\n')
    
def count_similarity(vec_list_1, vec_list_2):
    for vec_1 in vec_list_1:
        #print('1',vec_1)
        for vec_2 in vec_list_2:
            #print('2',vec_2)
            get_return = math_fuction_of_simi(vec_1, vec_2)
            #print(get_return)
            write_simi_in_txt(get_return)


    
#test_consine_result = 1 - spatial.distance.cosine(dataSetI, dataSetIII)
#test_consine_result
count_similarity(vector_1, vector_2)
f_txt.close()
print('done!')

C:\Users\Leaves\Anaconda3\lib\site-packages\scipy\spatial\distance.py:326: RuntimeWarning: invalid value encountered in true_divide
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


done!
